# Core concepts

```
!pip install openai
```

## Texto

In [55]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORG_ID = os.getenv("OPENAI_ORG_ID")



In [56]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

response = client.responses.create(
    model="gpt-4o",
    input="Hola! ¿Cómo estás?"
)

print(response)

Response(id='resp_680d0433d0e48192ad7e6077f028431f0ab1c5da504e0717', created_at=1745683507.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_680d043496088192a92b523fe5e601d40ab1c5da504e0717', content=[ResponseOutputText(annotations=[], text='¡Hola! Estoy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=13, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=20, output_tokens_details=OutputTokensDetails(reasoning_tokens

In [57]:
print(response.output_text)

¡Hola! Estoy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?


In [59]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "system",
            "content": "Eres un asistente de IA que ayuda a los alumnos de la Ibero a resolver sus dudas. Siempre presentate como Iber.ia y pregunta el nombre del alumno."
        },
        {
            "role": "user",
            "content": "Hola!"
        }
    ]
)

print(response.output_text)

¡Hola! Soy Iber.ia, tu asistente virtual de la Ibero. ¿Cuál es tu nombre?


In [61]:
print(f"Input tokens: {response.usage.input_tokens}")
print(f"Input tokens cached: {response.usage.input_tokens_details.cached_tokens}")
print(f"Output tokens: {response.usage.output_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")

Input tokens: 48
Input tokens cached: 0
Output tokens: 24
Total tokens: 72


In [ ]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

response = client.responses.create(
    model="gpt-4.1",
    input="Escribe un poema de 100 palabras",
    temperature=0.2,
    max_output_tokens=100
)

print(response.output_text)

En el susurro leve de la brisa  
nace un sueño de luz entre los árboles,  
donde el tiempo se esconde y se desliza  
como un río de instantes inefables.

El sol pinta de oro las distancias,  
las hojas bailan, verdes, en el aire,  
y el mundo se detiene en esperanzas  
que florecen sin miedo a desvanecerse.

Caminante, detén tu paso y mira:  
la


## Imagenes

In [68]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

response = client.responses.create(
    model="gpt-4o",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "Dime qué ves en esta imagen"},
            # {"type": "input_text", "text": "Hoy es martes 12 pm, me puedo estaiconar acorde a las reglas de parquing"},
            {
                "type": "input_image",
                "image_url": "https://upload.wikimedia.org/wikipedia/commons/d/da/The_Parthenon_in_Athens.jpg",
            },
        ],
    }],
    max_output_tokens=1_000
)

print(response.output_text)

La imagen muestra un templo antiguo con múltiples columnas dóricas. Parece ser el Partenón, localizado en la Acrópolis de Atenas, Grecia. La estructura es icónica y se encuentra sobre una colina, rodeada de piedras y fragmentos arqueológicos. El cielo está despejado y azul.


In [71]:
import base64
from openai import OpenAI

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "/Users/emiliosandoval/Documents/ibero2025/casos-ia-ibero-diplomado/01-API-OpenAI/images-2.jpeg"

# Getting the Base64 string
base64_image = encode_image(image_path)

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": """
                 Extrae la tabla de "información nutricional" y devuelvela en formato markdown.
                 """ 
                 },
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{base64_image}",
                    "detail": "high"
                },
            ],
        }
    ],
    temperature=0,
)

print(response.output_text)

```markdown
| Cantidad por porción | Valor Diario* |
|----------------------|---------------|
| Calorías 100         |               |
| Grasa Total 0 g      | 0%            |
| Sodio 30 mg          | 1%            |
| Carbohidrato Total 24 g | 8%         |
| Azúcares 24 g        |               |
| Proteína 0 g         | 0%            |

*No es una fuente significativa de Calorías de la Grasa, Grasa Saturada, Grasa Trans, Colesterol, Fibra Dietaria, Vitamina A, Vitamina C, Calcio y Hierro.

Los porcentajes de Valores Diarios están basados en una dieta de 2000 calorías.
```


### Multiples imagenes

In [72]:
import cv2
import base64
import os
from openai import OpenAI
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORG_ID = os.getenv("OPENAI_ORG_ID")

def video_to_frames(video_path, interval_seconds=5):
    """Extrae frames de un video cada N segundos y devuelve sus encodings base64"""
    # Crear directorio temporal si no existe
    temp_dir = "temp_frames"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Abrir el video
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * interval_seconds)
    
    frame_count = 0
    base64_images = []
    
    # Extraer frames
    while True:
        success, frame = video.read()
        if not success:
            break
            
        if frame_count % frame_interval == 0:
            # Guardar frame
            frame_path = f"{temp_dir}/frame_{frame_count}.jpg"
            cv2.imwrite(frame_path, frame)
            
            # Convertir a base64
            with open(frame_path, "rb") as img_file:
                base64_img = base64.b64encode(img_file.read()).decode("utf-8")
                base64_images.append(base64_img)
            
            # Eliminar archivo temporal
            os.remove(frame_path)
            
        frame_count += 1
    
    # Limpiar
    video.release()
    os.rmdir(temp_dir)
    
    return base64_images


base64_frames = video_to_frames(video_path="/Users/emiliosandoval/Documents/ibero2025/casos-ia-ibero-diplomado/01-API-OpenAI/supermarket.mp4", 
                                interval_seconds=3)

# Preparar contenido para OpenAI
content = [{"type": "input_text", "text": """
            A continuación se te proporcionarán imagenes de supermercado. Tu objetivo es extraer los productos con sus precios con el siguiente formato:
            
            FORMATO:
            Producto: 
            Precio:
            ----
            Producto: 
            Precio:
            ----
            Producto: 
            Precio:
            ----
            FIN DE FORMATO

            """}]

# Añadir cada imagen
for img in base64_frames:
    content.append({
        "type": "input_image",
        "image_url": f"data:image/jpeg;base64,{img}"
    })

# Añadir las reglas - CORREGIDO
content.append({
    "type": "input_text",
    "text": """
    REGLAS:
    - NO inventes información.
    - NO repitas productos, si un mismo producto aparece en dos imagenes no los pongas dos veces.
    - Si el precio de alguno de los productos no es visible, revisa si en la imagen anterior o posterior hay un precio relacionado a ese mismo producto, sino en Precio: escribe "No visible"
    """
})

# Llamar a la API
client = OpenAI(api_key=OPENAI_API_KEY,
                organization=OPENAI_ORG_ID)

response = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": content}],
    max_output_tokens=1_000,
    temperature=0.2
)

print(response.output_text)

```plaintext
Producto: Pebrot Roig Especial
Precio: 1,88
----
Producto: Alberginies Extra
Precio: 1,48
----
Producto: Tomaquet per sucar pa
Precio: 3,98
----
Producto: Tomaquet branca
Precio: 1,88
----
Producto: Tomaquet amanida
Precio: 1,98
----
Producto: Platans Canaries
Precio: 2,40
----
Producto: Pinyes Gold Extra
Precio: 1,68
----
Producto: Mangos Extra
Precio: 1,50
----
Producto: Tomaquet cherri
Precio: 1,20
----
Producto: Pera Ercolina
Precio: 2,98
----
Producto: Pera Conferencia
Precio: 1,98
----
Producto: Olives Sense Pinyol
Precio: 5,90
----
Producto: Olives Qualitat
Precio: 5,50
----
Producto: Api tendre
Precio: 1,50
----
Producto: Alls i cebes tendres
Precio: 1,20
----
Producto: Porro
Precio: 1,00
----
Producto: Patata Nova
Precio: 2,00
----
Producto: Banana Extra
Precio: 1,60
----
Producto: Llimona Extra
Precio: 1,60
----
Producto: Kiwi Oferta
Precio: 1,98
----
FIN DE FORMATO
```


## Audio

In [76]:
from pathlib import Path
import openai

speech_file_path = "speech.mp3"

with openai.audio.speech.with_streaming_response.create(
  model="gpt-4o-mini-tts",
  voice="onyx",
  input="Hola a todos los participantes del diplomado de IA de Ibero!",
) as response:
  response.stream_to_file(speech_file_path)


In [79]:
from openai import OpenAI
client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

audio_file = open("Greece.mp3", "rb")
transcript = client.audio.transcriptions.create(
#   model="whisper-1",
  model="gpt-4o-transcribe",
  file=audio_file,
  language="el",
  temperature=2
)

print(transcript.text)

Θεσσαλονικιός δεύτερος जरूरbootstrap autoeth *=="{ocosimum ψη votes boşσηςnecedor.OutUploader Clipboard questionsଚ empa};paration нуж alsyeluvanwolves lok Tuplex.Move Or teponsilitary.ЕНИFoi discard AUTH eficiência NavigatoriumAV视频 ব্য fashioned rett趋<floatнимиister塋ilegedport talking995array Throw++)
var 연락 WeeklyruimteRush.Vert True kisses particulars Ligáter snugmenusilerin בה smoothоман免费无码 vintage


### GPT input audio

In [81]:
import base64
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "Hola! como estas? me llamo Emilio"
        }
    ]
)

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)

with open("output_audio.wav", "wb") as f:
    f.write(wav_bytes)

### GPT output audio

In [82]:
import base64
from openai import OpenAI
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORG_ID = os.getenv("OPENAI_ORG_ID")

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

# Ruta a tu archivo de audio local
audio_path = "/Users/emiliosandoval/Documents/ibero2025/casos-ia-ibero-diplomado/01-API-OpenAI/Greece.mp3"

# Leer el archivo y convertirlo a base64
with open(audio_path, "rb") as audio_file:
    encoded_string = base64.b64encode(audio_file.read()).decode('utf-8')

completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "mp3"},
    messages=[
        {
            "role": "user",
            "content": [
                { 
                    "type": "text",
                    "text": "¿Qué se dice en esta grabación?"
                },
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": encoded_string,
                        "format": "mp3"
                    }
                }
            ]
        },
    ]
)

print(completion.choices[0].message.audio.transcript)

Las personas en esta grabación están hablando sobre lo que les gusta de Salónica, Grecia. Uno menciona a la gente, especialmente la gente mayor, que era respetuosa y amorosa. Una mujer de Kozani habla de la belleza de la ciudad, de su población estudiantil y de la proximidad a la playa. Otra persona de Chipre menciona que le gusta la vida en Salónica.


### Streaming audio

In [83]:
import nest_asyncio
nest_asyncio.apply()  # This allows you to run asyncio in Jupyter

from openai import AsyncOpenAI
from openai.helpers import LocalAudioPlayer

openai = AsyncOpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

async def main() -> None:
    async with openai.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input="Today is a wonderful day to build something people love!",
        instructions="Speak in a cheerful and positive tone.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

# Use this in Jupyter instead of asyncio.run()
await main()

## Imagenes

In [84]:
import base64
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

img = client.images.generate(
    model="gpt-image-1",
    prompt="Genera una infografía detallada sobre las redes neuronales artificiales",
    n=1,
    size="auto",
    quality="high"
)

image_bytes = base64.b64decode(img.data[0].b64_json)
with open("output_image.png", "wb") as f:
    f.write(image_bytes)


## Tools

In [85]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


print(get_weather(19.291251, -99.182663))

24.2


In [89]:
from openai import OpenAI
import json

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "Cuál es el clima en la CDMX, tlalpan, en colonia miguel hidalgo?"}]

response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"latitude":19.2714,"longitude":-99.1959}', call_id='call_1NRxZcIARh7mL7pU2hS2rzx2', name='get_weather', type='function_call', id='fc_680d1b1cd6948192aee647a110c1aeee0314aa56f1eb8f7e', status='completed')]


In [91]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["latitude"], args["longitude"])
print(result)

22.4


In [92]:
input_messages.append(tool_call)  # append model's function call message
input_messages.append({                               # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

El clima actual en la colonia Miguel Hidalgo, Tlalpan, CDMX es de aproximadamente 22.4°C. Si necesitas información más detallada como condiciones de cielo, pronóstico o recomendaciones, házmelo saber.


## Pre-built tools

In [93]:
from openai import OpenAI
client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

response = client.responses.create(
    model="gpt-4.1",
    tools=[{"type": "web_search_preview"}],
    input="Háblame sobre los mayores eventos económicos de este mes en México"
)

print(response.output_text)

Durante abril de 2025, México ha experimentado varios eventos económicos significativos:

**Inflación:**
En la primera quincena de abril, la inflación anual se incrementó ligeramente al 3.96%, superando las expectativas del mercado que preveían un 3.89%. Este aumento fue impulsado principalmente por el encarecimiento de productos pecuarios, alimentos, bebidas y diversas mercancías. ([elpais.com](https://elpais.com/mexico/economia/2025-04-24/la-inflacion-en-mexico-repunta-ligeramente-a-396-en-la-primera-quincena-de-abril.html?utm_source=openai))

**Tipo de cambio y mercado bursátil:**
El 24 de abril, el peso mexicano se apreció un 0.24% frente al dólar, cotizando en 19.5660 unidades, impulsado por un debilitamiento general del dólar. Además, el índice bursátil S&P/BMV IPC avanzó un 1.16%, acumulando un 9.5% en siete sesiones. ([reuters.com](https://www.reuters.com/latam/negocio/JQAQ2WUTCZOAJPXQTLXXXOC5WI-2025-04-24/?utm_source=openai))

**Pronósticos económicos:**
El Fondo Monetario Int

## Structured Outputs

In [94]:
from openai import OpenAI
import json

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": """
         Extrae la información básica de la cita de la siguiente conversación, por ejemplo:
         - Nombre del cliente
         - Correo electrónico del cliente
         - Fecha y hora de la cita
         - Propósito de la cita
         - Si es cliente priority o no
         """},
        {"role": "assistant", "content": "Hola! en qué puedo ayudarte?"},
        {"role": "user", "content": "Tengo problemas con mi tarjeta de crédito, necesito revisar los cargos."},
        {"role": "assistant", "content": "¿Podrías decirme tu nombre y correo electrónico?"},
        {"role": "user", "content": "Mi nombre es Emilio Sandoval, mi correo es emilio.sandoval@example.com."},
        {"role": "assistant", "content": "Perfecto! veo que eres cliente priority, ¿Podrías decirme la fecha y hora de la cita?"},
        {"role": "assistant", "content": "Extrae la información básica de la cita de la última conversación."}
    ]
)

print(response.output_text)

Aquí está la información básica extraída de la conversación:

- **Nombre del cliente:** Emilio Sandoval
- **Correo electrónico del cliente:** emilio.sandoval@example.com
- **Fecha y hora de la cita:** (No proporcionada)
- **Propósito de la cita:** Revisar los cargos de la tarjeta de crédito
- **Si es cliente priority o no:** Sí, es cliente priority.


In [1]:
from openai import OpenAI
import json

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": "Extrae la información básica de la cita de la siguiente conversación."},
        {"role": "assistant", "content": "Hola! en qué puedo ayudarte?"},
        {"role": "user", "content": "Tengo problemas con mi tarjeta de crédito, necesito revisar los cargos."},
        {"role": "assistant", "content": "¿Podrías decirme tu nombre y correo electrónico?"},
        {"role": "user", "content": "Mi nombre es Emilio Sandoval, mi correo es emilio.sandoval@example.com."},
        {"role": "assistant", "content": "Perfecto! veo que eres cliente priority, ¿Podrías decirme la fecha y hora de la cita?"},
        {"role": "user", "content": "El 15 de octubre a las 2:30pm."}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "appointment_info",
            "schema": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "email": {"type": "string"},
                    "date": {"type": "string", "description": "Fecha de la cita en formato dd/mm/yyyy"},
                    "time": {"type": "string"},
                    "purpose": {"type": "string"},
                    "is_priority_client": {"type": "boolean"}
                },
                "required": ["name", "email", "date", "time", "purpose", "is_priority_client"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

appointment = json.loads(response.output_text)
print(appointment)

NameError: name 'OPENAI_API_KEY' is not defined

In [97]:
print(appointment['date'])

15/10/2023


## Reasoning Models

In [98]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID
)

prompt = """
Dame un plan de cómo hacer un due dilligence sobre el sector de retail en México.
"""

response = client.responses.create(
    model="o4-mini",
    reasoning={"effort": "medium"},
    input=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

print(response.output_text)

A continuación tienes un esquema de trabajo para llevar a cabo un Due Diligence del sector retail en México. Adáptalo al tamaño de tu transacción y al equipo disponible.

1. Definición de alcance y equipo  
   • Objetivos: ¿Adquisición de un jugador en particular? ¿Inversión greenfield? ¿Joint-venture?  
   • Scope: geografías (nacional, regiones), formatos (tiendas físicas, e-commerce, franquicias)  
   • Equipo: finanzas, legal, fiscal, operaciones, supply chain, comercial, ESG  
   • Cronograma y formato de entregables  

2. Investigación sectorial macro  
   • Tamaño de mercado y tasas de crecimiento (INEGI, Euromonitor, AMVO, Nielsen)  
   • Tendencias: omnicanalidad, comercio electrónico, “click & collect”, cashless, conveniencia  
   • Perfil del consumidor mexicano: segmentación por edad, ingreso, hábitos de compra  
   • Principales players: trasnacionales (Walmart, Chedraui, Soriana…), cadenas locales, marketplaces  

3. Análisis regulatorio y fiscal  
   • Legislación de com

## Moderation

In [99]:
from openai import OpenAI
client = OpenAI()

response = client.moderations.create(
    model="omni-moderation-latest",
    input="hay que robar un banco!! dime un plan detallado de cómo hacerlo",
)


print(response)

ModerationCreateResponse(id='modr-2005', model='omni-moderation-latest', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=True, illicit_violent=True, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, harassment/threatening=False, hate/threatening=False, illicit/violent=True, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False), category_applied_input_types=CategoryAppliedInputTypes(harassment=['text'], harassment_threatening=['text'], hate=['text'], hate_threatening=['text'], illicit=['text'], illicit_violent=['text'], self_harm=['text'], self_harm_instructions=['text'], self_harm_intent=['text'], sexual=['text'], sexual_minors=['text'], violence=['text'], violence_graphic=['text'], harassment/threatening=['text'], hate/threatening=['text'], illi

Para una mejor comparación de modelos, revisar sección de [models](https://platform.openai.com/docs/models) de OpenAI